In [1]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [2]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [3]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [4]:
# # replace NAN's
# def rmNAN(df, col):
#     df.col = df.col.fillna(0)

In [5]:
sarahMetrics = loadMetricData('sarahYee', "metrics")
sarahMetrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-04 23:55:00,71,0,0.9,0.000067,93.2,93.2
1436,2016-05-04 23:56:00,69,0,1.0,0.000067,94.1,94.1
1437,2016-05-04 23:57:00,68,0,0.9,0.000069,94.1,94.1
1438,2016-05-04 23:58:00,66,0,1.0,0.000071,94.1,94.1
1439,2016-05-04 23:59:00,65,0,0.9,0.000067,94.1,94.1


In [6]:
sarahMetrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [7]:
# clean metric data

sarahMetrics.timestamp = pd.to_datetime(sarahMetrics.timestamp)

sarahMetrics.heartrate = sarahMetrics.heartrate.fillna(0)
sarahMetrics.steps = sarahMetrics.steps.fillna(0)
sarahMetrics.calories = sarahMetrics.calories.fillna(0)
sarahMetrics.gsr = sarahMetrics.gsr.fillna(0)
sarahMetrics.skintemp = sarahMetrics.skintemp.fillna(0)
sarahMetrics.airtemp = sarahMetrics.airtemp.fillna(0)

sarahMetrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
0,2016-04-22 00:00:00,0,0,0,0,0,0
1,2016-04-22 00:01:00,0,0,0,0,0,0
2,2016-04-22 00:02:00,0,0,0,0,0,0
3,2016-04-22 00:03:00,0,0,0,0,0,0
4,2016-04-22 00:04:00,0,0,0,0,0,0


In [8]:
sarahActivities = loadMetricData('sarahYee', "activities")
sarahActivities.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,type,actual seconds,steps,calories,minutes,heart rate avg,heart rate min,heart rate max,state,version,id
2,5/4/16 19:19,2016-05-04T19:19:07Z,America/New_York,-240,5/4/16 19:29,2016-05-04T19:29:20Z,America/New_York,-240,walk,380,904,17.400000,10.216667,109.357895,NaN,NaN,complete,1,walk__2016-05-04T191907
3,5/4/16 19:52,2016-05-04T19:52:24Z,America/New_York,-240,5/4/16 19:57,2016-05-04T19:57:53Z,America/New_York,-240,walk,235,533,7.100000,5.483333,96.454544,NaN,NaN,complete,1,walk__2016-05-04T195224
4,5/4/16 20:26,2016-05-04T20:26:49Z,America/New_York,-240,5/4/16 20:34,2016-05-04T20:34:34Z,America/New_York,-240,walk,188,345,6.400000,7.750000,106.696808,NaN,NaN,complete,1,walk__2016-05-04T202649
5,5/4/16 20:38,2016-05-04T20:38:37Z,America/New_York,-240,5/4/16 20:44,2016-05-04T20:44:05Z,America/New_York,-240,walk,184,340,16.200001,5.466667,133.668472,NaN,NaN,complete,1,walk__2016-05-04T203837
6,5/5/16 0:48,2016-05-05T00:48:54Z,America/New_York,-240,5/5/16 0:55,2016-05-05T00:55:25Z,America/New_York,-240,walk,311,725,14.000000,6.516667,103.729904,NaN,NaN,complete,1,walk__2016-05-05T004854


In [9]:
sarahSleep = loadMetricData('sarahYee', "sleep")
sarahSleep.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,light mins,deep mins,...,toss turns,type,actual seconds,calories,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-05-01 04:45:50,2016-05-01T04:45:50Z,America/New_York,-240,2016-05-01 12:43:50,2016-05-01T12:43:50Z,America/New_York,-240,219,124,...,32,sleep,28080,482.799988,67.618942,NaN,NaN,complete,2,sleep__2016-05-01T044550
0,2016-05-02 04:35:21,2016-05-02T04:35:21Z,America/New_York,-240,2016-05-02 10:51:21,2016-05-02T10:51:21Z,America/New_York,-240,195,105,...,26,sleep,22560,333.364014,66.956131,NaN,NaN,complete,2,sleep__2016-05-02T043521
0,2016-05-03 03:43:04,2016-05-03T03:43:04Z,America/New_York,-240,2016-05-03 10:17:04,2016-05-03T10:17:04Z,America/New_York,-240,200,77,...,36,sleep,22980,362.299988,70.224983,NaN,NaN,complete,2,sleep__2016-05-03T034304
0,2016-05-04 03:35:37,2016-05-04T03:35:37Z,America/New_York,-240,2016-05-04 11:30:37,2016-05-04T11:30:37Z,America/New_York,-240,223,148,...,34,sleep,28500,448.600006,69.207352,NaN,NaN,complete,2,sleep__2016-05-04T033537
0,2016-05-04 11:50:37,2016-05-04T11:50:37Z,America/New_York,-240,2016-05-04 12:22:37,2016-05-04T12:22:37Z,America/New_York,-240,31,0,...,1,sleep,1920,30.400000,70.055222,NaN,NaN,complete,2,sleep__2016-05-04T115037


In [10]:
sarahSleep.dtypes

start time              object
start time ISO          object
start time timezone     object
start time offset        int64
end time                object
end time ISO            object
end time timezone       object
end time offset          int64
light mins               int64
deep mins                int64
rem mins                 int64
interruption mins        int64
unknown mins             int64
interruptions            int64
toss turns               int64
type                    object
actual seconds           int64
calories               float64
heart rate avg         float64
heart rate min         float64
heart rate max         float64
state                   object
version                  int64
id                      object
dtype: object

In [11]:
sarahJournal = loadMetricData('sarahYee', "journal")
sarahJournal.tail(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
122,2016-05-03 15:45:00,2016-05-03 16:00:00,5/3/16,3:45:00 PM,4:00:00 PM,sitting with sally,5th floor cage,calm,NaN,NaN,NaN,4,2,NaN,NaN,NaN,5,NaN
123,2016-05-03 16:30:00,2016-05-03 17:30:00,5/3/16,4:30:00 PM,5:30:00 PM,getting mothers day cards/dads bday card,out and about,tired,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,5,NaN
124,2016-05-03 18:30:00,2016-05-03 18:40:00,5/3/16,6:30:00 PM,6:40:00 PM,realized i forgot to get something to bring fo...,out and about,tired,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,5,NaN
125,2016-05-03 07:40:00,2016-05-03 20:00:00,5/3/16,7:40:00 AM,8:00:00 PM,nap,home,tired,NaN,NaN,NaN,NaN,4,4,NaN,NaN,5,NaN
126,2016-05-03 20:48:00,2016-05-03 09:15:00,5/3/16,8:48:00 PM,9:15:00 AM,exercise (shorter than usual... tired),home,dying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
sarahJournal.dtypes

starttime       object
endtime         object
date            object
start           object
end             object
activity        object
location        object
generalemo      object
qualitative     object
excited        float64
happy          float64
calm           float64
anxious        float64
sad            float64
angry          float64
hungry         float64
tired          float64
bored          float64
dtype: object

In [13]:
# clean metric data
sarahJournal.starttime= pd.to_datetime(sarahJournal.starttime)
sarahJournal.endtime= pd.to_datetime(sarahJournal.endtime)


# sarahJournal.qualitative = sarahJournal.qualitative.convert_objects(convert_numeric=True)
# sarahJournal.excited = sarahJournal.excited.convert_objects(convert_numeric=True)
# sarahJournal.happy = sarahJournal.happy.convert_objects(convert_numeric=True)

sarahJournal.qualitative = sarahJournal.qualitative.fillna(0)
sarahJournal.excited = sarahJournal.excited.fillna(0)
sarahJournal.happy = sarahJournal.happy.fillna(0)
sarahJournal.calm = sarahJournal.calm.fillna(0)
sarahJournal.anxious = sarahJournal.anxious.fillna(0)
sarahJournal.sad = sarahJournal.sad.fillna(0)
sarahJournal.angry = sarahJournal.angry.fillna(0)
sarahJournal.hungry = sarahJournal.hungry.fillna(0)
sarahJournal.tired = sarahJournal.tired.fillna(0)
sarahJournal.bored = sarahJournal.bored.fillna(0)

print(sarahJournal.shape)
sarahJournal.head(5)
# sarahJournal.tail(5)

(127, 18)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,0,0,0,0,0,0,0,0
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,0,0,0,0,0,0,0,0
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,0,0,0,0,0,0,0,0
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,3-May,0,0,0,0,0,0,0,0,0
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,0,0,0,0,0,0,0,0


In [14]:

sarahJournal['boolexcited'] =  sarahJournal['excited'] > 0
sarahJournal['boolhappy'] =  sarahJournal['happy'] > 0
sarahJournal['boolcalm'] =  sarahJournal['calm'] > 0
sarahJournal['boolanxious'] =  sarahJournal['anxious'] > 0
sarahJournal['boolsad'] =  sarahJournal['sad'] > 0
sarahJournal['boolangry'] =  sarahJournal['angry'] > 0
sarahJournal['booltired'] =  sarahJournal['tired'] > 0
sarahJournal['boolhungry'] =  sarahJournal['hungry'] > 0
sarahJournal['boolbored'] =  sarahJournal['bored'] > 0


for i in range(0, sarahJournal.shape[0]):
#     sarahJournal['intqual'] = int(sarahJournal.iloc[i, 8])
    sarahJournal['intexcited'] = int(sarahJournal.iloc[i, 9])
    sarahJournal['inthappy'] = int(sarahJournal.iloc[i, 10]) 
    sarahJournal['intcalm'] = int(sarahJournal.iloc[i, 11])
    sarahJournal['intanxious'] = int(sarahJournal.iloc[i, 12])
    sarahJournal['intsad'] = int(sarahJournal.iloc[i, 13]) 
    sarahJournal['intangry'] = int(sarahJournal.iloc[i, 14]) 
    sarahJournal['inttired'] = int(sarahJournal.iloc[i, 15]) 
    sarahJournal['inthungry'] = int(sarahJournal.iloc[i, 16]) 
    sarahJournal['intbored'] = int(sarahJournal.iloc[i, 17]) 
    
sarahJournal['exercise'] = sarahJournal['activity'].str.contains('exercise')
sarahJournal['thesis'] = sarahJournal['activity'].str.contains('thesis')
sarahJournal['sally'] = sarahJournal['activity'].str.contains('sally')
sarahJournal['barb'] = sarahJournal['activity'].str.contains('barb')
sarahJournal['sol'] = sarahJournal['activity'].str.contains('sol')
sarahJournal['arash'] = sarahJournal['activity'].str.contains('arash')

sarahJournal['home'] = sarahJournal['location'].str.contains('home')
sarahJournal['work'] = sarahJournal['location'].str.contains("barb's office")




boolexcited = sarahJournal.iloc[:, 18].values.reshape(sarahJournal.shape[0], 1)
boolhappy = sarahJournal.iloc[:, 19].values.reshape(sarahJournal.shape[0], 1)
boolcalm = sarahJournal.iloc[:, 20].values.reshape(sarahJournal.shape[0], 1)
boolanxious = sarahJournal.iloc[:, 21].values.reshape(sarahJournal.shape[0], 1)
boolsad = sarahJournal.iloc[:, 22].values.reshape(sarahJournal.shape[0], 1)
boolangry = sarahJournal.iloc[:, 23].values.reshape(sarahJournal.shape[0], 1)
booltired = sarahJournal.iloc[:, 24].values.reshape(sarahJournal.shape[0], 1)
boolhungry = sarahJournal.iloc[:, 25].values.reshape(sarahJournal.shape[0], 1)
boolbored = sarahJournal.iloc[:, 26].values.reshape(sarahJournal.shape[0], 1)
intexcited = sarahJournal.iloc[:, 27].values.reshape(sarahJournal.shape[0], 1)
inthappy = sarahJournal.iloc[:, 28].values.reshape(sarahJournal.shape[0], 1)
intcalm = sarahJournal.iloc[:, 29].values.reshape(sarahJournal.shape[0], 1)
intanxious = sarahJournal.iloc[:, 30].values.reshape(sarahJournal.shape[0], 1)
intsad = sarahJournal.iloc[:, 31].values.reshape(sarahJournal.shape[0], 1)
intangry = sarahJournal.iloc[:, 32].values.reshape(sarahJournal.shape[0], 1)
inttired = sarahJournal.iloc[:, 33].values.reshape(sarahJournal.shape[0], 1)
inthungry = sarahJournal.iloc[:, 34].values.reshape(sarahJournal.shape[0], 1)
intbored = sarahJournal.iloc[:, 35].values.reshape(sarahJournal.shape[0], 1)
exercise = sarahJournal.iloc[:, 36].values.reshape(sarahJournal.shape[0], 1)
thesis = sarahJournal.iloc[:, 37].values.reshape(sarahJournal.shape[0], 1)
sally = sarahJournal.iloc[:, 38].values.reshape(sarahJournal.shape[0], 1)
barb = sarahJournal.iloc[:, 39].values.reshape(sarahJournal.shape[0], 1)
sol = sarahJournal.iloc[:, 40].values.reshape(sarahJournal.shape[0], 1)
arash = sarahJournal.iloc[:, 41].values.reshape(sarahJournal.shape[0], 1)
work = sarahJournal.iloc[:, 42].values.reshape(sarahJournal.shape[0], 1)
home = sarahJournal.iloc[:, 43].values.reshape(sarahJournal.shape[0], 1)


sarahJournal.head(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,inthungry,intbored,exercise,thesis,sally,barb,sol,arash,home,work
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,...,0,0,False,True,False,False,False,False,False,True
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,...,0,0,False,True,False,False,False,False,False,True
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,...,0,0,False,False,False,False,False,False,False,False
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,3-May,0,...,0,0,False,False,False,False,False,False,False,False
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,...,0,0,False,False,False,False,False,False,False,False


In [15]:
# sarahJournal.dtypes

In [16]:
# sarahJournal.excited.convert_objects(convert_numeric=True)

In [17]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [18]:
# indicator for walking
totalData = makeIndicators('excitement', sarahMetrics, sarahJournal, 0, 1, 18)
totalData = makeIndicators('happy', sarahMetrics, sarahJournal, 0, 1, 19)
totalData = makeIndicators('calm', sarahMetrics, sarahJournal, 0, 1, 20)
totalData = makeIndicators('anxious', sarahMetrics, sarahJournal, 0, 1, 21)
totalData = makeIndicators('sad', sarahMetrics, sarahJournal, 0, 1, 22)
totalData = makeIndicators('angry', sarahMetrics, sarahJournal, 0, 1, 23)
totalData = makeIndicators('hungry', sarahMetrics, sarahJournal, 0, 1, 24)
totalData = makeIndicators('tired', sarahMetrics, sarahJournal, 0, 1, 25)
totalData = makeIndicators('bored', sarahMetrics, sarahJournal, 0, 1, 26)

totalData = makeIndicators('exercise', sarahMetrics, sarahJournal, 0, 1, 36)
totalData = makeIndicators('thesis', sarahMetrics, sarahJournal, 0, 1, 37)
totalData = makeIndicators('sally', sarahMetrics, sarahJournal, 0, 1, 38)
totalData = makeIndicators('barb', sarahMetrics, sarahJournal, 0, 1, 39)
totalData = makeIndicators('sol', sarahMetrics, sarahJournal, 0, 1, 40)
totalData = makeIndicators('arash', sarahMetrics, sarahJournal, 0, 1, 41)
totalData = makeIndicators('work', sarahMetrics, sarahJournal, 0, 1, 42)
totalData = makeIndicators('home', sarahMetrics, sarahJournal, 0, 1, 43)

In [19]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  18540.000000  18540.000000  18540.000000  18540.000000  18540.000000   
mean      81.698058      8.188080      1.737449      0.042450     82.748549   
std       28.134934     25.363245      8.760419      0.403378     23.151163   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       69.000000      0.000000      1.000000      0.000053     86.000000   
50%       85.000000      0.000000      1.100000      0.000061     88.700000   
75%       98.000000      0.000000      1.700000      0.000078     91.400000   
max      168.000000    695.000000   1167.400020     10.500000     97.700000   

            airtemp excitement      happy      calm   anxious    ...     \
count  18540.000000      18540      18540     18540     18540    ...      
mean      79.519180  0.0108414  0.0307443  0.135761  0.197843    ...      
std       22.483904   0.103559   0.172629  0.342543  0.398383  

In [20]:
print(totalData.head())

            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-22 00:00:00          0      0         0    0         0        0   
1 2016-04-22 00:01:00          0      0         0    0         0        0   
2 2016-04-22 00:02:00          0      0         0    0         0        0   
3 2016-04-22 00:03:00          0      0         0    0         0        0   
4 2016-04-22 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm  ...    tired  bored exercise thesis  sally   barb  \
0      False  False  False  ...    False  False    False  False  False  False   
1      False  False  False  ...    False  False    False  False  False  False   
2      False  False  False  ...    False  False    False  False  False  False   
3      False  False  False  ...    False  False    False  False  False  False   
4      False  False  False  ...    False  False    False  False  False  False   

     sol  arash   work   home  
0  False  False  F

In [21]:
totalData.to_csv('totsarah.csv')

In [22]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,tired,bored,exercise,thesis,sally,barb,sol,arash,work,home
0,2016-04-22 00:00:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-04-22 00:01:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-04-22 00:02:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-04-22 00:03:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-04-22 00:04:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(18540, 28)
            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-22 00:00:00          0      0         0    0         0        0   
1 2016-04-22 00:01:00          0      0         0    0         0        0   
2 2016-04-22 00:02:00          0      0         0    0         0        0   
3 2016-04-22 00:03:00          0      0         0    0         0        0   
4 2016-04-22 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm ...  sally   barb    sol  arash   work   home  \
0      False  False  False ...  False  False  False  False  False  False   
1      False  False  False ...  False  False  False  False  False  False   
2      False  False  False ...  False  False  False  False  False  False   
3      False  False  False ...  False  False  False  False  False  False   
4      False  False  False ...  False  False  False  False  False  False   

  weekday month hour min  
0       4     4    0   0  
1       4     

In [24]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
thesis = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
sally = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
barb = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
sol = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
arash = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
work = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
home = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 27].values.reshape(totalData.shape[0], 1)

In [25]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
        exercise, 
        thesis, 
        sally, 
        barb, 
        sol, 
        arash, 
        work, 
        home, 
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(18540, 17)

In [26]:
# define y
y = totalData.excitement.values
y.shape

(18540,)

In [27]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [28]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [29]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

# Excited Model Results

In [30]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0111727538912 0.0100683207479
 Accuracy Rate  0.988827246109  0.989931679252
True Positives             0.0             0.0
True Negatives              --             1.0


(0.010068320747932399, 0.9899316792520676, 0.0, 1.0)

In [31]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0101710587147 0.0109672779576
      Accuracy Rate  0.989828941285  0.989032722042
     True Positives   0.11724137931 0.0892857142857
     True Negatives              --  0.998183799491


(0.010967277957569219,
 0.98903272204243076,
 0.089285714285714288,
 0.99818379949146385)

In [32]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0376791493296 0.0334412081985
 Accuracy Rate   0.96232085067  0.966558791802
True Positives  0.344827586207  0.303571428571
True Negatives              --  0.973301852525


(0.033441208198489752,
 0.96655879180151028,
 0.30357142857142855,
 0.97330185252451873)

In [33]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0101710587147 0.0104279036318
 Accuracy Rate  0.989828941285  0.989572096368
True Positives   0.11724137931 0.0892857142857
True Negatives              --  0.998728659644


(0.010427903631787127,
 0.98957209636821286,
 0.089285714285714288,
 0.99872865964402469)

In [34]:
# define y
y = totalData.happy.values
y.shape

(18540,)

In [35]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [36]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Happy Model Results

In [37]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0488519032208 0.0548363897878
 Accuracy Rate  0.951148096779  0.945163610212
True Positives  0.281725888325  0.244318181818
True Negatives              --  0.968065354623


(0.0548363897878461,
 0.94516361021215389,
 0.24431818181818182,
 0.96806535462309695)

In [38]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0302049622438  0.032182668105
      Accuracy Rate  0.969795037756  0.967817331895
     True Positives 0.0177664974619 0.0227272727273
     True Negatives              --    0.9987003342


(0.032182668104998202,
 0.96781733189500185,
 0.022727272727272728,
 0.99870033419977722)

In [39]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0376791493296 0.0400934915498
 Accuracy Rate   0.96232085067   0.95990650845
True Positives 0.0406091370558 0.0511363636364
True Negatives              --  0.989602673598


(0.040093491549802227,
 0.95990650845019776,
 0.05113636363636364,
 0.98960267359821763)

In [40]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0308984435198  0.032182668105
 Accuracy Rate   0.96910155648  0.967817331895
True Positives  0.010152284264 0.0170454545455
True Negatives              --  0.998886000743


(0.032182668104998202,
 0.96781733189500185,
 0.017045454545454544,
 0.9988860007426662)

In [41]:
# define y
y = totalData.calm.values
y.shape

(18540,)

In [42]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [43]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Calm Model Results

In [44]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.114193250116 0.123336929162
 Accuracy Rate 0.885806749884 0.876663070838
True Positives 0.354262672811  0.33290653009
True Negatives             --  0.96548839155


(0.12333692916217188,
 0.87666307083782813,
 0.33290653008962867,
 0.96548839154988497)

In [45]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.588380335953 0.588457389428
      Accuracy Rate 0.411619664047 0.411542610572
     True Positives  0.94297235023 0.950064020487
     True Negatives             -- 0.323572474378


(0.5884573894282632,
 0.4115426105717368,
 0.95006402048655569,
 0.32357247437774522)

In [46]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.147865618739  0.15300251708
 Accuracy Rate 0.852134381261  0.84699748292
True Positives 0.570276497696 0.556978233035
True Negatives             -- 0.894373562016


(0.15300251708018697,
 0.84699748291981303,
 0.55697823303457106,
 0.89437356201631457)

In [47]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate  0.129603945138  0.138259618842
 Accuracy Rate  0.870396054862  0.861740381158
True Positives 0.0357142857143 0.0217669654289
True Negatives              --  0.998954193683


(0.13825961884214311,
 0.86174038115785689,
 0.02176696542893726,
 0.99895419368332983)

In [48]:
# define y
y = totalData.anxious.values
y.shape

(18540,)

In [49]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [50]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Anxious Model Results

In [51]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.174294960703 0.181769147789
 Accuracy Rate 0.825705039297 0.818230852211
True Positives 0.540519276161 0.495559502664
True Negatives             -- 0.900135256988


(0.18176914778856526,
 0.81823085221143477,
 0.49555950266429838,
 0.90013525698827768)

In [52]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.256356911697 0.268428622798
      Accuracy Rate 0.743643088303 0.731571377202
     True Positives  0.76435877262 0.719360568384
     True Negatives             -- 0.734670874662


(0.26842862279755486,
 0.73157137720244514,
 0.71936056838365892,
 0.73467087466185754)

In [53]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.199414393589 0.204782452355
 Accuracy Rate 0.800585606411 0.795217547645
True Positives 0.520849724626 0.497335701599
True Negatives             -- 0.870829576195


(0.2047824523552679,
 0.79521754764473207,
 0.49733570159857904,
 0.87082957619477008)

In [54]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.151564185545 0.161452714851
 Accuracy Rate 0.848435814455 0.838547285149
True Positives 0.335169158143 0.299289520426
True Negatives             -- 0.975428313796


(0.16145271485077312,
 0.83854728514922683,
 0.29928952042628776,
 0.97542831379621275)

In [55]:
# define y
y = totalData.sad.values
y.shape

(18540,)

In [56]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [57]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Sad Model Values

In [58]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0085529357374 0.00845019777059
 Accuracy Rate  0.991447064263   0.991549802229
True Positives  0.990494296578   0.986547085202
True Negatives              --   0.991758756321


(0.0084501977705861205,
 0.99154980222941391,
 0.98654708520179368,
 0.99175875632140853)

In [59]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00762829403606 0.00773103200288
      Accuracy Rate   0.992371705964   0.992268967997
     True Positives   0.990494296578   0.986547085202
     True Negatives               --   0.992507960292


(0.007731032002876663,
 0.99226896799712339,
 0.98654708520179368,
 0.99250796029218957)

In [60]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0198027431037 0.0212153901474
 Accuracy Rate  0.980197256896  0.978784609853
True Positives  0.969581749049  0.964125560538
True Negatives              --  0.979396890804


(0.021215390147428983,
 0.97878460985257099,
 0.9641255605381166,
 0.97939689080352121)

In [61]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0189551548775 0.0190578928443
 Accuracy Rate  0.981044845123  0.980942107156
True Positives  0.555133079848  0.560538116592
True Negatives              --  0.998501592058


(0.019057892844300612,
 0.98094210715569941,
 0.5605381165919282,
 0.99850159205843791)

In [62]:
# define y
y = totalData.angry.values
y.shape

(18540,)

In [63]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [64]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Angry Model Results

In [65]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0510864539991 0.0537576411363
 Accuracy Rate  0.948913546001  0.946242358864
True Positives  0.509114583333  0.517520215633
True Negatives              --  0.976883066846


(0.053757641136281911,
 0.9462423588637181,
 0.51752021563342321,
 0.97688306684646509)

In [66]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set      Test Set   
------------------- -------------- ---------------
         Error Rate 0.067575897673 0.0713772024452
      Accuracy Rate 0.932424102327  0.928622797555
     True Positives 0.505208333333  0.514824797844
     True Negatives             --  0.958196879214


(0.071377202445163615,
 0.92862279755483634,
 0.51482479784366575,
 0.9581968792140243)

In [67]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0633379565418 0.0702984537936
 Accuracy Rate  0.936662043458  0.929701546206
True Positives  0.100260416667   0.11590296496
True Negatives              --  0.987863610094


(0.070298453793599419,
 0.92970154620640055,
 0.11590296495956873,
 0.98786361009439416)

In [68]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0574818924333 0.0641855447681
 Accuracy Rate  0.942518107567  0.935814455232
True Positives 0.0598958333333  0.078167115903
True Negatives              --  0.997110383356


(0.064185544768069036,
 0.93581445523193096,
 0.078167115902964962,
 0.99711038335580815)

In [69]:
# define y
y = totalData.hungry.values
y.shape

(18540,)

In [70]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [71]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Hungry Model Results

In [72]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set     Test Set   
-------------- --------------- --------------
    Error Rate 0.0585606410849 0.059331175836
 Accuracy Rate  0.941439358915 0.940668824164
True Positives    0.4046021093 0.397408207343
True Negatives              -- 0.989998038831


(0.059331175836030203,
 0.9406688241639698,
 0.39740820734341253,
 0.98999803883114335)

In [73]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0650331329943 0.0658036677454
      Accuracy Rate  0.934966867006  0.934196332255
     True Positives  0.250239693193  0.246220302376
     True Negatives              --  0.996666012944


(0.065803667745415323,
 0.93419633225458465,
 0.24622030237580994,
 0.99666601294371449)

In [74]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0788257050393 0.0821646889608
 Accuracy Rate  0.921174294961  0.917835311039
True Positives  0.411313518696  0.403887688985
True Negatives              --  0.964502843695


(0.082164688960805463,
 0.91783531103919458,
 0.4038876889848812,
 0.96450284369484207)

In [75]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0799044536909 0.0827040632866
 Accuracy Rate  0.920095546309  0.917295936713
True Positives 0.0105465004794 0.0107991360691
True Negatives              --  0.999607766229


(0.082704063286587554,
 0.91729593671341247,
 0.01079913606911447,
 0.99960776622867231)

In [76]:
# define y
y = totalData.tired.values
y.shape

(18540,)

In [77]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [78]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Tired Model Results

In [79]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00924641701341 0.00970873786408
 Accuracy Rate   0.990753582987   0.990291262136
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0097087378640776691, 0.99029126213592233, 0.0, 1.0)

In [80]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0119432886423 0.0118662351672
      Accuracy Rate  0.988056711358  0.988133764833
     True Positives 0.0166666666667 0.0555555555556
     True Negatives              --  0.997276688453


(0.011866235167206042,
 0.98813376483279391,
 0.055555555555555552,
 0.99727668845315909)

In [81]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00932347048852 0.00970873786408
 Accuracy Rate   0.990676529511   0.990291262136
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0097087378640776691, 0.99029126213592233, 0.0, 1.0)

In [82]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00924641701341 0.00970873786408
 Accuracy Rate   0.990753582987   0.990291262136
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0097087378640776691, 0.99029126213592233, 0.0, 1.0)

In [83]:
# define y
y = totalData.bored.values
y.shape

(18540,)

In [84]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [85]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Bored Model Results

In [86]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0510864539991 0.0582524271845
 Accuracy Rate  0.948913546001  0.941747572816
True Positives             0.0             0.0
True Negatives              --  0.999809123879


(0.058252427184466021, 0.94174757281553401, 0.0, 0.99980912387860277)

In [87]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set   
------------------- ---------------- ---------------
         Error Rate  0.0526275235013 0.0596907587199
      Accuracy Rate   0.947372476499   0.94030924128
     True Positives 0.00151975683891             0.0
     True Negatives               --  0.998282114907


(0.059690758719884933, 0.94030924128011506, 0.0, 0.99828211490742513)

In [88]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0507011866235 0.0580726357425
 Accuracy Rate  0.949298813376  0.941927364257
True Positives             0.0             0.0
True Negatives              --             1.0


(0.058072635742538653, 0.94192736425746137, 0.0, 1.0)

In [89]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate  0.0517799352751 0.0598705501618
 Accuracy Rate   0.948220064725  0.940129449838
True Positives 0.00151975683891             0.0
True Negatives               --  0.998091238786


(0.059870550161812294, 0.94012944983818769, 0.0, 0.9980912387860279)